In [1]:
#!pip install transformers pandas torch sklearn

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np





/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
# Load the datasets
train_data = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/train.csv')
test_data = pd.read_csv('/kaggle/input/kaggle-llm-science-exam/test.csv')



In [3]:
# Preprocess the text data
train_data['text'] = train_data['prompt'] + ' ' + train_data['A'] + ' ' + train_data['B'] + ' ' + train_data['C'] + ' ' + train_data['D'] + ' ' + train_data['E']
test_data['text'] = test_data['prompt'] + ' ' + test_data['A'] + ' ' + test_data['B'] + ' ' + test_data['C'] + ' ' + test_data['D'] + ' ' + test_data['E']

# Label encoding for the target variable
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['answer'])



In [4]:
# Initialize the BERT tokenizer and model
model_dir = '/kaggle/input/huggingface-bert/bert-large-uncased'

#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained(model_dir)
model = BertForSequenceClassification.from_pretrained(model_dir, num_labels=5)



Some weights of the model checkpoint at /kaggle/input/huggingface-bert/bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [5]:
# Tokenize the text data and convert to PyTorch tensors
train_encodings = tokenizer(list(train_data['text']), truncation=True, padding=True, max_length=512)
train_inputs = torch.tensor(train_encodings['input_ids'])
train_masks = torch.tensor(train_encodings['attention_mask'])
train_labels = torch.tensor(list(train_data['label']))
test_encodings = tokenizer(list(test_data['text']), truncation=True, padding=True, max_length=512)
test_inputs = torch.tensor(test_encodings['input_ids'])
test_masks = torch.tensor(test_encodings['attention_mask'])



In [6]:
# Create a DataLoader for the training and test data
#train_data = TensorDataset(train_inputs, train_masks, train_labels)
#train_sampler = RandomSampler(train_data)
#train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=8)
#test_data = TensorDataset(test_inputs, test_masks)
#test_sampler = SequentialSampler(test_data)
#test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=8)

In [7]:
# Create a DataLoader for the training and test data
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=8)

test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=8)


In [8]:
# Specify the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)  # Decrease from 2e-5 to 1e-5

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [9]:
# Specify the device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [10]:
# Move the model to the device
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,

In [11]:
# Training loop
model.train()
for epoch in range(3):  
    for batch in train_dataloader:
        b_input_ids, b_input_mask, b_labels = batch
        
        # Move the batch tensors to the same device as the model
        b_input_ids = b_input_ids.to(device)
        b_input_mask = b_input_mask.to(device)
        b_labels = b_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 │   │   b_labels = b_labels.to(device)                                                      │
│   11 │   │                                                                                       │
│   12 │   │   optimizer.zero_grad()                                                               │
│ ❱ 13 │   │   outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, l    │
│   14 │   │   loss = outputs[0]                                                                   │
│   15 │   │   loss.backward()                                                                     │
│   16 │   │   optimizer.step()                                                                    │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/models/bert/modeling_bert.py:1562 in        │
│ forward                                                                                          │
│                                                                                                  │
│   1559 │   │   """                                                                               │
│   1560 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│   1561 │   │                                                                                     │
│ ❱ 1562 │   │   outputs = self.bert(                                                              │
│   1563 │   │   │   input_ids,                                                                    │
│   1564 │   │   │   attention_mask=attention_mask,                                                │
│   1565 │   │   │   token_type_ids=token_type_ids,                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks

In [ ]:
# Move the model back to CPU for inference
#model.to('cpu')

In [ ]:
# Evaluation
model.eval()
predictions = []
for batch in test_dataloader:
    b_input_ids, b_input_mask = batch
    
    # Move the batch tensors to the same device as the model
    b_input_ids = b_input_ids.to(device)
    b_input_mask = b_input_mask.to(device)
    
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    predictions.append(logits)

# Prepare the submission dataframe
test_predictions = [item for sublist in predictions for item in sublist]
test_predictions = le.inverse_transform(np.argmax(test_predictions, axis=1))
submission = pd.DataFrame({
    'id': test_data['id'],
    'prediction': test_predictions
})

# Save the submission dataframe as a CSV file
submission.to_csv('/kaggle/working/submission.csv', index=False)